In [53]:
import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine 
import pandas as pd
import numpy as np

In [54]:
import sqlalchemy
sqlalchemy.__version__

'1.4.46'

In [87]:
# Function to extract table to a pandas DataFrame
def extract_table_to_pandas(tablename, db_engine):
    query = "SELECT * FROM {}".format(tablename)
    return pd.read_sql(query, db_engine)

db_engine = create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                                   format('subnet_user', 'subnet_user123', 
                                                          'localhost', 'elearning'))

In [88]:
def print_user_comparison(user1, user2, user3):
    print("Course id overlap between users:")
    print("================================")
    print("User 1 and User 2 overlap: {}".format(
    set(user1.course_id) & set(user2.course_id)
    ))
    print("User 1 and User 3 overlap: {}".format(
    set(user1.course_id) & set(user3.course_id)
    ))
    print("User 2 and User 3 overlap: {}".format(
    set(user2.course_id) & set(user3.course_id)
    ))

In [89]:
# Get user with id 4387
user1 = pd.read_sql("SELECT * FROM rating where user_id=4387", db_engine)

# Get user with id 18163
user2 = pd.read_sql("SELECT * FROM rating where user_id=18163", db_engine)

# Get user with id 8770
user3 = pd.read_sql("SELECT * FROM rating where user_id=8770", db_engine)

# Use the helper function to compare the 3 users
print_user_comparison(user1, user2, user3)

Course id overlap between users:
User 1 and User 2 overlap: {32, 96, 36, 6, 7, 44, 95}
User 1 and User 3 overlap: set()
User 2 and User 3 overlap: set()


In [90]:
def extract_rating_data(db_engine):
    return pd.read_sql("SELECT * FROM rating", db_engine)

def extract_course_data(db_engine):
    return pd.read_sql("SELECT * FROM courses", db_engine)

In [91]:
ratings = extract_rating_data(db_engine)

course_data = extract_course_data(db_engine)

In [92]:
ratings.head()

,user_id,course_id,rating
0,1,6,4
1,1,36,5
2,1,37,5
3,1,45,5
4,1,50,5


In [93]:
course_data.head()

,course_id,title,description,programming_language
0,1,Machine Learning with Apache Spark,"Spark is a powerful, general purpose tool for ...",python
1,2,Financial Analytics in Spreadsheets,Monitoring the evolution of traded assets is k...,spreadsheets
2,3,Intermediate R,The intermediate R course is the logical next ...,r
3,4,Data Visualization with ggplot2 (Part 2),This ggplot2 tutorial builds on your knowledge...,r
4,5,Fraud Detection in R,The Association of Certified Fraud Examiners e...,r


In [94]:
ratings.user_id.nunique()

38174

In [95]:
course_data.shape

(100, 4)

In [96]:
# Print out the number of missing values per column
print(course_data.isnull().sum())

course_id               0
title                   0
description             0
programming_language    3
dtype: int64


In [97]:
# The transformation should fill in the missing values
def transform_fill_programming_language(course_data):
    imputed = course_data.fillna({"programming_language": "R"})
    return imputed

transformed = transform_fill_programming_language(course_data)

# Print out the number of missing values per column of transformed
print(transformed.isnull().sum())

course_id               0
title                   0
description             0
programming_language    0
dtype: int64


In [98]:
# transformation function
def transform_avg_rating(rating_data):
    # Group by course_id and extract average rating per course
    avg_rating = rating_data.groupby('course_id').rating.mean()
    # Return sorted average ratings per course
    sort_rating = avg_rating.sort_values(ascending=False).reset_index()
    return sort_rating

In [99]:
# Use transform_avg_rating on the extracted data and print results
avg_course_ratings = transform_avg_rating(ratings)
print(avg_course_ratings) 

    course_id    rating
0          46  4.800000
1          23  4.800000
2          96  4.692765
3          56  4.661765
4          24  4.653061
..        ...       ...
94         54  4.238095
95         92  4.222222
96         29  4.208333
97         17  4.147059
98         42  4.107570

[99 rows x 2 columns]


In [100]:
def get_courses_to_recommend(ratings, course_data):
    user_list = ratings.user_id.unique()

    course_set = set(course_data.course_id.unique())

    non_enrolled_users = []

    for user in user_list:
        rated_course_set = set(ratings[ratings.user_id == user].course_id.unique())
        non_enrolled_courses = course_set - rated_course_set
        non_enrolled_users = non_enrolled_users + [(user, cid) for cid in non_enrolled_courses]
        if(user == 10):
            break
    non_enrolled_courses_df = pd.DataFrame(non_enrolled_users, columns = ['user_id', 'course_id'])
    return non_enrolled_courses_df

courses_to_recommend = get_courses_to_recommend(ratings, course_data)
courses_to_recommend.to_csv('courses_to_recommend.csv', index = False)
# courses_to_recommend.sample(n=50).to_csv('courses_to_recommend_small.csv', index = False)

In [101]:
course_data.shape

(100, 4)

In [102]:
temp = pd.read_csv('courses_to_recommend.csv')
temp.sample(n=900).to_csv('courses_to_recommend_small.csv', index = False)

In [103]:
temp.shape

(955, 2)

In [104]:
courses_to_recommend = pd.read_csv('courses_to_recommend_small.csv')

courses_to_recommend.head()

,user_id,course_id
0,1,4
1,10,74
2,2,37
3,1,44
4,5,12


In [105]:
courses_to_recommend['course_duration'] = np.random.randint(2, 11, size=len(courses_to_recommend))

In [106]:
courses_to_recommend.groupby('user_id').count()

,course_id,course_duration
user_id,,
1,88,88
2,93,93
3,79,79
4,95,95
5,87,87
6,94,94
7,92,92
8,91,91
9,88,88


In [107]:
# Complete the transformation function
def transform_recommendations(avg_course_ratings, courses_to_recommend):
    # Merge both DataFrames
    merged = courses_to_recommend.merge(avg_course_ratings) 
    
    # Sort values by rating and group by user_id
    grouped = merged.sort_values("rating", ascending=False).groupby("user_id")
    
    # Produce the top 3 values and sort by user_id
    recommendations = grouped.head(5).sort_values("user_id").reset_index()
    
    final_recommendations = recommendations[["user_id", "course_id","rating","course_duration"]]
    
    # Return final recommendations
    return final_recommendations


# Use the function with the predefined DataFrame objects
recommendations = transform_recommendations(avg_course_ratings, courses_to_recommend)

In [108]:
recommendations.head()

,user_id,course_id,rating,course_duration
0,1,24,4.653061,10
1,1,96,4.692765,2
2,1,56,4.661765,2
3,1,46,4.800000,5
4,1,23,4.800000,2


In [109]:
# Merge both DataFrames
merged = courses_to_recommend.merge(avg_course_ratings) 

merged.head()

,user_id,course_id,course_duration,rating
0,1,4,6,4.305233
1,4,4,6,4.305233
2,3,4,2,4.305233
3,6,4,6,4.305233
4,5,4,10,4.305233


In [110]:
merged.sort_values("rating", ascending=False).head()

,user_id,course_id,course_duration,rating
515,3,46,2,4.8
520,7,46,10,4.8
345,9,23,8,4.8
344,4,23,2,4.8
343,3,23,9,4.8


In [113]:
merged[merged.user_id == 7]

,user_id,course_id,course_duration,rating
6,7,4,6,4.305233
13,7,74,3,4.529412
23,7,37,8,4.581818
29,7,44,8,4.593866
41,7,12,7,4.340909
...,...,...,...,...
843,7,90,6,4.492191
850,7,78,9,4.433333
862,7,26,4,4.646259
870,7,50,7,4.456914


In [114]:
# Sort values by rating and group by user_id
grouped = merged.sort_values("rating", ascending=False).groupby("user_id")


# Produce the top 3 values and sort by user_id
recommendations = grouped.head(3).sort_values("user_id")

recommendations.head()

,user_id,course_id,course_duration,rating
341,1,23,2,4.800000
429,1,96,2,4.692765
516,1,46,5,4.800000
432,2,96,6,4.692765
338,2,23,7,4.800000


In [116]:
recommendations[recommendations.user_id == 9]

,user_id,course_id,course_duration,rating
345,9,23,8,4.800000
355,9,56,3,4.661765
521,9,46,5,4.800000


In [118]:
recommendations = transform_recommendations(avg_course_ratings, courses_to_recommend)

In [123]:
recommendations.head(10)

,user_id,course_id,rating,course_duration
0,1,24,4.653061,10
1,1,96,4.692765,2
2,1,56,4.661765,2
3,1,46,4.800000,5
4,1,23,4.800000,2
5,2,23,4.800000,7
6,2,46,4.800000,10
7,2,56,4.661765,9
8,2,24,4.653061,2
9,2,96,4.692765,6


In [133]:


def load_to_dwh(recommendations, db_engine):
    recommendations.to_sql("recommendations", db_engine, if_exists="replace")

In [134]:
load_to_dwh(recommendations, db_engine)

In [135]:
def recommendations_for_user(user_id, threshold , programming_language, course_duration):
    # Join with the courses table
    query = """
    SELECT title, rating FROM recommendations 
    INNER JOIN courses ON courses.course_id = recommendations.course_id
    WHERE user_id=%(user_id)s AND rating>%(threshold)s AND programming_language = %(programming_language)s AND course_duration<%(course_duration)s;
    """
    # Add the threshold parameter
    predictions_df = pd.read_sql(query, db_engine, params = {"user_id": user_id, 
                                                           "threshold": threshold, 
                                                            "programming_language":programming_language,
                                                            'course_duration':course_duration
                                                            })
    return predictions_df.title.values

# Try the function you created

print(recommendations_for_user(4, 3, "r",8))

['GARCH Models in R' 'Nonlinear Modeling in R with GAMs']


## ETL: Combine all the operations

In [128]:
def etl(db_engine):
    # Extract the data
    courses = extract_course_data(db_engine)
    rating = extract_rating_data(db_engine)
    
    # Clean up courses data
    courses = transform_fill_programming_language(courses)
    
    # Get the average course ratings
    avg_course_rating = transform_avg_rating(rating)
    
    # Get eligible user and course id pairs
    courses_to_recommend = pd.read_csv('courses_to_recommend_small.csv')
    
    # Calculate the recommendations
    recommendations = transform_recommendations(
    avg_course_rating,
    courses_to_recommend,
    )
    
    # Load the recommendations into the database
    load_to_dwh(recommendations, db_engine)

In [130]:
etl(db_engine)

In [141]:
a = int(input('Enter user id:'))
b = float(input('Enter minimum rating of course:'))
c = str(input('Enter language:'))
d = int(input('Enter the maximum length of preferred language course(in hrs) : '))

print(recommendations_for_user(a,b,c,d)) 

Enter user id:4
Enter minimum rating of course:3
Enter language:r
Enter the maximum length of preferred language course(in hrs) : 8
['GARCH Models in R' 'Nonlinear Modeling in R with GAMs']


## Assignment

1. Recommend courses based on programming_language.
   - Build a logic and create flolwchart of it.
   - Write the code for it in python/sql.